In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Read in data
schedule=pd.read_excel('../data/Marshall_Course_Enrollment_1516_1617.xlsx')
capacities=pd.read_excel('../data/Marshall_Room_Capacity_Chart.xlsx')

data = (pd.merge(schedule, capacities, how = 'left', left_on= 'First Room', right_on='Room')
       [['Term','Section','Department','First Begin Time','First Days','First End Time','First Room',
         'Reg Count','Size']]
       )
# clean building not included in capacitites
data = data[data['Size'].notnull()]

# calculate utility
data = data.rename(columns={'First Begin Time':'Begin','First Days':'Days',
                            'First End Time':'End','First Room':'Room','Reg Count':'Registered'})
data = data.assign(Utility=lambda data: data.Registered/data.Size)

data.head()

,Term,Section,Department,Begin,Days,End,Room,Registered,Size,Utility
1,20153,14025,ACCT,08:00:00,MW,09:50:00,ACC303,24,46.0,0.521739
2,20153,14026,ACCT,10:00:00,MW,11:50:00,ACC303,40,46.0,0.869565
3,20153,14027,ACCT,12:00:00,MW,13:50:00,ACC303,42,46.0,0.913043
5,20153,14040,ACCT,08:00:00,TH,09:50:00,ACC303,29,46.0,0.630435
6,20153,14042,ACCT,12:00:00,TH,13:50:00,ACC303,40,46.0,0.869565


In [50]:
def convert(inputTime):
    # define convert function
    try:
        hh,mm,ss=str(inputTime).split(':')
        ans=int(hh)+int(mm)/60+int(ss)/3600
    except:
        ans=np.nan
    return ans

data['Begin'].apply(convert(data.Begin), axis = 1)
data.head()

TypeError: 'float' object is not callable

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Read in data
schedule=pd.read_excel('../data/Marshall_Course_Enrollment_1516_1617.xlsx')
capacities=pd.read_excel('../data/Marshall_Room_Capacity_Chart.xlsx')



def convert(inputTime):
    # define convert function
    try:
        hh,mm,ss=str(inputTime).split(':')
        ans=int(hh)+int(mm)/60+int(ss)/3600
    except:
        ans=np.nan
    return ans


def loadDataDict(df,roomSet):
    # adding entries to a list instead of finding beginning and end, and adding empty list for unused classrooms
    ans={}
    # Start with empty lists in all classrooms
    terms=[20153,20161,20162,20163,20171,20172]
    for term in terms:
        for room in roomSet:
            for day in 'MTWHF':
                ans[term,dept,day]=[]
    for index,row in df.iterrows():   
        term=row['Term']                       # Obtain the corresponding column of each row
        dept=row['Departments']
        days=row['First Days'] 
        beg=convert(row['First Begin Time'])   # Convert the begin time strings into decimal numbers using challenge 1
        end=convert(row['First End Time'])     # Convert the begin time strings into decimal numbers using challenge 1
        # Skip rows in which beg and end are np.nan (not a number), and in which the room is not in the capacity file
        #import pdb; pdb.set_trace()
        if np.isnan(beg) or np.isnan(end) or room not in roomSet:  
            continue     # Command to skip this iteration of the loop
        for day in 'MTWHF':   # Iterate through the sequence ['M','T','W','H','F']
            if day in days: 
                ans[term,room,day].append([beg,end])
    
    return ans
                    
def computeUsage(inputList, primeStart,primeEnd):
    # sorting the inputList
    sortedList=sorted(inputList)
    usage=0
    prev=0
    for start,end in sortedList:
        if end<primeStart:
            continue
        if start>primeEnd:
            break
        start=max(prev,start)
        end=max(prev,end)
        overlap=max(0,min(primeEnd,end)-max(primeStart,start))
        usage+=overlap
        prev=end
    return usage/(primeEnd-primeStart)

# Beginning of main code.
primeStart=10
primeEnd=16

# Read in data
schedule=pd.read_excel('../data/Marshall_Course_Enrollment_1516_1617.xlsx')
cancelled=pd.read_excel('../data/Cancelled_Courses_1516_1617.xlsx')
master=schedule.append(cancelled)
capacities=pd.read_excel('../data/Marshall_Room_Capacity_Chart.xlsx')

# Set rooms to focus on to be those in the capacity file.
roomSet=set(capacities.Room)

# Load the data from the master DataFrame into a dictionary of the format in challenge 2
dataDict=loadDataDict(master,roomSet)

# Create a list of lists, corresponding to the data we want to dump out. 
lines=[]
for term,room,day in loadDataDict(master,roomSet):
    # Each row of the output data has columns being term, room, day, utilization
    lines.append([term,room,day,computeUsage(dataDict[term,room,day],primeStart,primeEnd)])

# Store data back into a dataframe
output=pd.DataFrame(lines,columns=['Term','Room','Day','Utilization'])

# Output to a file
output.to_csv('RoomUsage.csv')
output.tail()

,Course,Course Prefix,Course Suffix,Department,First Begin Time,First Days,First End Time,First Instructor,First Instructor UID,First Room,...,Second Begin Time,Second Days,Second End Time,Second Instructor,Second Instructor UID,Second Room,Section,Session,Term,Title
3227,NaN,GSBA,519B,NaN,18:00:00,MW,22:00:00,"Voigt, Carl, W",8.225711e+09,JKP210,...,NaN,NaN,NaN,NaN,NaN,NaN,15644,868,20172,Strategic Formulation for Competitive Advantage
3228,NaN,GSBA,581,NaN,18:00:00,M,22:00:00,"Selby, Richard",3.550296e+09,JKP212,...,NaN,NaN,NaN,NaN,NaN,NaN,16008,865,20172,Information Management and Analytics
3229,NaN,GSBA,534,NaN,18:00:00,W,22:00:00,"Yormark, Jonathan, Seth",7.791420e+09,JKP212,...,NaN,NaN,NaN,NaN,NaN,NaN,15787,869,20172,Operations Management
3230,NaN,GSBA,563B,NaN,08:50:00,FS,16:30:00,"Porter, Dawn",4.812435e+09,NaN,...,NaN,NaN,NaN,"Bhambri, Arvind",7.043327e+09,NaN,15915,628,20172,Technology and Information Systems Management
3231,NaN,GSBA,564,NaN,08:50:00,FS,16:30:00,"Porter, Dawn",4.812435e+09,NaN,...,NaN,NaN,NaN,"Bhambri, Arvind",7.043327e+09,NaN,15917,629,20172,Functional Strategies and Implementation


In [7]:
cancelled.tail()

,Course,Course Prefix,Course Suffix,Department,First Begin Time,First Days,First End Time,First Instructor,First Instructor UID,First Room,...,Second Begin Time,Second Days,Second End Time,Second Instructor,Second Instructor UID,Second Room,Section,Session,Term,Title
3227,NaN,GSBA,519B,NaN,18:00:00,MW,22:00:00,"Voigt, Carl, W",8.225711e+09,JKP210,...,NaN,NaN,NaN,NaN,NaN,NaN,15644,868,20172,Strategic Formulation for Competitive Advantage
3228,NaN,GSBA,581,NaN,18:00:00,M,22:00:00,"Selby, Richard",3.550296e+09,JKP212,...,NaN,NaN,NaN,NaN,NaN,NaN,16008,865,20172,Information Management and Analytics
3229,NaN,GSBA,534,NaN,18:00:00,W,22:00:00,"Yormark, Jonathan, Seth",7.791420e+09,JKP212,...,NaN,NaN,NaN,NaN,NaN,NaN,15787,869,20172,Operations Management
3230,NaN,GSBA,563B,NaN,08:50:00,FS,16:30:00,"Porter, Dawn",4.812435e+09,NaN,...,NaN,NaN,NaN,"Bhambri, Arvind",7.043327e+09,NaN,15915,628,20172,Technology and Information Systems Management
3231,NaN,GSBA,564,NaN,08:50:00,FS,16:30:00,"Porter, Dawn",4.812435e+09,NaN,...,NaN,NaN,NaN,"Bhambri, Arvind",7.043327e+09,NaN,15917,629,20172,Functional Strategies and Implementation
